Goal: Create Dataset with data from every pixel over time

In [8]:
import os
import glob
import rasterio
import numpy as np
import pandas as pd
from rasterio.enums import Resampling
from rasterio.warp import reproject
import re

###############################################################################
# 1. Use the revised detect_dataset_type and extract_year
###############################################################################

def detect_dataset_type(file_path):
    name = os.path.basename(file_path)
    name_lower = name.lower()

    if "population_" in name_lower:
        return "pop"
    elif "ndvi_" in name_lower:
        return "ndvi"
    elif "lst_" in name_lower:
        return "lst"
    elif "_gp" in name_lower:
        return "gpp"
    elif "lct" in name_lower:
        return "landcover"
    elif re.search(r'(20\d{2})r\.tif$', name_lower):
        return "precip"
    else:
        return None

def extract_year(file_path):
    import re
    name = os.path.basename(file_path)
    match = re.search(r'(20\d{2})', name)
    if match:
        return int(match.group(1))
    return None

###############################################################################
# 2. Search folders for TIFs
###############################################################################
base_folder = "Datasets_Hackathon 1/Datasets_Hackathon"  # Adjust if needed
all_tifs = glob.glob(os.path.join(base_folder, "**/*.tif"), recursive=True)

files_dict = {
    "landcover": {},
    "ndvi": {},
    "lst": {},
    "pop": {},
    "gpp": {},
    "precip": {}
}

for fp in all_tifs:
    dset_type = detect_dataset_type(fp)
    if not dset_type:
        continue
    year = extract_year(fp)
    if not year:
        continue
    files_dict[dset_type][year] = fp

# For years 2021-2023, copy population from 2020 if missing
for y in [2021, 2022, 2023]:
    if y not in files_dict["pop"] and 2020 in files_dict["pop"]:
        files_dict["pop"][y] = files_dict["pop"][2020]

###############################################################################
# 3. Pick a Land Cover raster to define resolution/extent
###############################################################################
# We'll try 2010, or any other year available.
if 2010 in files_dict["landcover"]:
    ref_path = files_dict["landcover"][2010]
else:
    lc_years = sorted(files_dict["landcover"].keys())
    if lc_years:
        ref_path = files_dict["landcover"][lc_years[0]]
    else:
        raise RuntimeError("No land cover files found!")

with rasterio.open(ref_path) as src_ref:
    ref_data = src_ref.read(1)
    ref_profile = src_ref.profile
    ref_transform = src_ref.transform
    ref_crs = src_ref.crs
    ref_height, ref_width = ref_data.shape

###############################################################################
# 4. Helper to resample
###############################################################################
def resample_to_reference(source_path, cap_gpp=False):
    with rasterio.open(source_path) as src:
        arr_src = src.read(1)
        transform_src = src.transform
        crs_src = src.crs
        
        dest_data = np.zeros((ref_height, ref_width), dtype=np.float32)

        reproject(
            arr_src,
            dest_data,
            src_transform=transform_src,
            src_crs=crs_src,
            dst_transform=ref_transform,
            dst_crs=ref_crs,
            resampling=Resampling.bilinear
        )

        # Cap GPP if needed
        if cap_gpp:
            dest_data[dest_data > 4000] = 0
        
        return dest_data

###############################################################################
# 5. Load and resample all data for 2010-2023
###############################################################################
years = range(2010, 2024)
data_by_type = {k: {} for k in files_dict.keys()}

for y in years:
    # landcover
    if y in files_dict["landcover"]:
        data_by_type["landcover"][y] = resample_to_reference(files_dict["landcover"][y])
    else:
        data_by_type["landcover"][y] = None
    
    # ndvi
    if y in files_dict["ndvi"]:
        data_by_type["ndvi"][y] = resample_to_reference(files_dict["ndvi"][y])
    else:
        data_by_type["ndvi"][y] = None

    # lst
    if y in files_dict["lst"]:
        data_by_type["lst"][y] = resample_to_reference(files_dict["lst"][y])
    else:
        data_by_type["lst"][y] = None

    # pop
    if y in files_dict["pop"]:
        data_by_type["pop"][y] = resample_to_reference(files_dict["pop"][y])
    else:
        data_by_type["pop"][y] = None

    # gpp
    if y in files_dict["gpp"]:
        data_by_type["gpp"][y] = resample_to_reference(files_dict["gpp"][y], cap_gpp=True)
    else:
        data_by_type["gpp"][y] = None

    # precip
    if y in files_dict["precip"]:
        data_by_type["precip"][y] = resample_to_reference(files_dict["precip"][y])
    else:
        data_by_type["precip"][y] = None

###############################################################################
# 6. Assemble into a DataFrame
###############################################################################
rows = []
for y in years:
    lc_arr   = data_by_type["landcover"][y]
    if lc_arr is None:
        # skip if no landcover for this year
        continue
    
    # flatten landcover
    lc_flat = lc_arr.flatten()
    
    # flatten other variables or fill with NaN
    ndvi_arr = data_by_type["ndvi"][y]
    ndvi_flat = ndvi_arr.flatten() if ndvi_arr is not None else np.full(lc_flat.shape, np.nan)
    
    lst_arr = data_by_type["lst"][y]
    lst_flat = lst_arr.flatten() if lst_arr is not None else np.full(lc_flat.shape, np.nan)

    pop_arr = data_by_type["pop"][y]
    pop_flat = pop_arr.flatten() if pop_arr is not None else np.full(lc_flat.shape, np.nan)

    gpp_arr = data_by_type["gpp"][y]
    gpp_flat = gpp_arr.flatten() if gpp_arr is not None else np.full(lc_flat.shape, np.nan)

    prc_arr = data_by_type["precip"][y]
    prc_flat = prc_arr.flatten() if prc_arr is not None else np.full(lc_flat.shape, np.nan)

    # Make a DataFrame for this year
    temp_df = pd.DataFrame({
        f"LC_{y}": lc_flat,
        f"NDVI_{y}": ndvi_flat,
        f"LST_{y}": lst_flat,
        f"POP_{y}": pop_flat,
        f"GPP_{y}": gpp_flat,
        f"PRECIP_{y}": prc_flat,
        "pixel_id": np.arange(len(lc_flat))
    })
    rows.append(temp_df)

# Merge across all years on pixel_id => wide table
df_full = None
for year_df in rows:
    if df_full is None:
        df_full = year_df
    else:
        df_full = df_full.merge(year_df, on="pixel_id", how="outer")

###############################################################################
# 7. Exclude invalid land cover classes -128 and 255
###############################################################################
lc_cols = [c for c in df_full.columns if c.startswith("LC_")]
mask_invalid = np.zeros(len(df_full), dtype=bool)
for c in lc_cols:
    mask_invalid |= (df_full[c] == -128) | (df_full[c] == 255)

df_filtered = df_full[~mask_invalid].copy()

print("Final DataFrame shape:", df_filtered.shape)
print(df_filtered.head())

# Save to CSV if desired
df_filtered.to_csv("final_pixel_dataframe.csv", index=False)
print("DataFrame saved to final_pixel_dataframe.csv")


Final DataFrame shape: (168212, 85)
      LC_2010    NDVI_2010   LST_2010  POP_2010  GPP_2010   PRECIP_2010  \
475      16.0  1004.347412  40.922581  0.006259       0.0 -3.402823e+38   
1039     16.0  1082.418701  41.056068  0.005568       0.0 -3.312761e+38   
1040     16.0  1006.241638  40.807373  0.005553       0.0 -3.402823e+38   
1603     16.0  1091.514160  40.763355  0.005525       0.0 -3.164094e+38   
1604     16.0  1015.872864  40.485077  0.005415       0.0 -3.300985e+38   

      pixel_id  LC_2011    NDVI_2011   LST_2011  ...   LST_2022  POP_2022  \
475        475     16.0  1036.330200  41.960651  ...  39.550652  0.007126   
1039      1039     16.0  1075.744751  42.358791  ...  39.756775  0.005950   
1040      1040     16.0  1008.242249  41.900337  ...  39.534119  0.005174   
1603      1603     16.0  1065.525269  42.219372  ...  39.943565  0.005401   
1604      1604     16.0   994.906250  41.864021  ...  39.667461  0.004754   

      GPP_2022   PRECIP_2022  LC_2023    NDVI_2023